In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=de2f4da28a9f843a5774a03bc07f3f0e00230c0d98cf5ccc2f6061c728e1451e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [62]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, sum
# Add a unique identifier column to both dataframes
from pyspark.sql.functions import monotonically_increasing_id


In [2]:

# Create a Spark session
spark = SparkSession.builder \
    .appName("ReorderDataFrameB") \
    .getOrCreate()


# Sắp xếp dataframe PCA theo kết quả cluster (DHC)

In [3]:
FILE_PATHA = "/content/pca_data.csv"
FILE_PATHB = "/content/div_var_result.csv"

In [15]:
dfA = spark.read.csv(FILE_PATHA, header=True, inferSchema=True)
dfB = spark.read.csv(FILE_PATHB, header=True, inferSchema=True)

In [183]:
dfA_ordered = dfA.withColumn("id", monotonically_increasing_id())

In [184]:
dfA_ordered.show()

+------------------+--------------------+-------------------+---+
|               PC1|                 PC2|                PC3| id|
+------------------+--------------------+-------------------+---+
|      -4.688990299|        -0.161059582|       -2.150476907|  0|
|        2.84303513|        -0.197028684|  2.047757826841097|  1|
|      -2.508281353|  0.8519421916573712| 0.1633727577626868|  2|
|2.6546251428541296|  1.5369548861577003|       -0.179903563|  3|
|0.5523046199295245|0.049763351943901864|       -0.461802933|  4|
|      -0.637442564|         -1.26089685|        0.737590415|  5|
|      -0.570561647|        -1.558762645|       -0.854124396|  6|
|2.3958492591218414|  0.8991361813361359|       -1.496119463|  7|
|2.9725888006414487|   1.231300543824599|       -0.944586686|  8|
| 5.042755804708938|         -0.58937187|       -0.647846936|  9|
|1.7261751586803042|  2.6121183140028776|       -1.642682194| 10|
|      -3.898882282|  1.9486847297964611| 0.7244371358938236| 11|
|1.7889389

In [212]:
dfB_ordered = dfB.join(dfA_ordered,
                       (dfB.col1 == dfA_ordered.PC1) &
                       (dfB.col2 == dfA_ordered.PC2) &
                       (dfB.col3 == dfA_ordered.PC3),
                       'inner').orderBy('id')
dfB_ordered.show()

+------------------+--------------------+-------------------+-------+------------------+--------------------+-------------------+---+
|              col1|                col2|               col3|cluster|               PC1|                 PC2|                PC3| id|
+------------------+--------------------+-------------------+-------+------------------+--------------------+-------------------+---+
|      -4.688990299|        -0.161059582|       -2.150476907|      0|      -4.688990299|        -0.161059582|       -2.150476907|  0|
|        2.84303513|        -0.197028684|  2.047757826841097|      1|        2.84303513|        -0.197028684|  2.047757826841097|  1|
|      -2.508281353|  0.8519421916573712| 0.1633727577626868|      0|      -2.508281353|  0.8519421916573712| 0.1633727577626868|  2|
|2.6546251428541296|  1.5369548861577003|       -0.179903563|      3|2.6546251428541296|  1.5369548861577003|       -0.179903563|  3|
|0.5523046199295245|0.049763351943901864|       -0.461802933| 

In [213]:
dfB_ordered = dfB_ordered.dropDuplicates(['id'])
dfB_ordered.show()

+------------------+--------------------+-------------------+-------+------------------+--------------------+-------------------+---+
|              col1|                col2|               col3|cluster|               PC1|                 PC2|                PC3| id|
+------------------+--------------------+-------------------+-------+------------------+--------------------+-------------------+---+
|      -4.688990299|        -0.161059582|       -2.150476907|      0|      -4.688990299|        -0.161059582|       -2.150476907|  0|
|        2.84303513|        -0.197028684|  2.047757826841097|      1|        2.84303513|        -0.197028684|  2.047757826841097|  1|
|      -2.508281353|  0.8519421916573712| 0.1633727577626868|      0|      -2.508281353|  0.8519421916573712| 0.1633727577626868|  2|
|2.6546251428541296|  1.5369548861577003|       -0.179903563|      3|2.6546251428541296|  1.5369548861577003|       -0.179903563|  3|
|0.5523046199295245|0.049763351943901864|       -0.461802933| 

In [214]:
result = dfB_ordered.select ('PC1', 'PC2', 'PC3', 'cluster')
result.show()

+------------------+--------------------+-------------------+-------+
|               PC1|                 PC2|                PC3|cluster|
+------------------+--------------------+-------------------+-------+
|      -4.688990299|        -0.161059582|       -2.150476907|      0|
|        2.84303513|        -0.197028684|  2.047757826841097|      1|
|      -2.508281353|  0.8519421916573712| 0.1633727577626868|      0|
|2.6546251428541296|  1.5369548861577003|       -0.179903563|      3|
|0.5523046199295245|0.049763351943901864|       -0.461802933|      2|
|      -0.637442564|         -1.26089685|        0.737590415|      2|
|      -0.570561647|        -1.558762645|       -0.854124396|      2|
|2.3958492591218414|  0.8991361813361359|       -1.496119463|      3|
|2.9725888006414487|   1.231300543824599|       -0.944586686|      3|
| 5.042755804708938|         -0.58937187|       -0.647846936|      1|
|1.7261751586803042|  2.6121183140028776|       -1.642682194|      3|
|      -3.898882282|

In [215]:
out = "/content/result.csv"
result.write.csv(out, header = True)
print("Lưu thành công!!!")

Lưu thành công!!!


# Sắp xếp dataframe PCA theo kết quả cluster (AHC)